In [1]:
import os
import sys
import time
import copy
import h5py
import numpy as np
from scipy.misc import imread, imresize
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML

from visualize_utils import *

In [2]:
# download InceptionV5 model if not
if not os.path.exists("./inception5h.zip"):
    os.system("python -m wget -o ./inception5h.zip https://storage.googleapis.com/download.tensorflow.org/models/inception5h.zip && unzip inception5h.zip")

In [3]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)
# importing InceptionV5 model
with tf.gfile.FastGFile('tensorflow_inception_graph.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
t_input = tf.placeholder(np.float32, name='input') # define the input tensor
imagenet_mean = 117.0
t_preprocessed = t_input-imagenet_mean
tf.import_graph_def(graph_def, {'input':t_preprocessed})

In [4]:
layers = [op.name for op in graph.get_operations() if op.type=='Conv2D' and 'import/' in op.name]
feature_nums = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]

print('Number of layers', len(layers))
print('Total number of feature channels:', sum(feature_nums))


# Helper functions for TF Graph visualization

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = tf.compat.as_bytes("<stripped %d bytes>"%size)
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# Visualizing the network graph. Be sure expand the "mixed" nodes to see their 
# internal structure. We are going to visualize "Conv2D" nodes.
tmp_def = rename_nodes(graph_def, lambda s:"/".join(s.split('_',1)))
show_graph(tmp_def)

('Number of layers', 59)
('Total number of feature channels:', 7548)


In [5]:
# reading sample image
im = np.expand_dims(imresize(imread(os.path.join("./sample_images", "images.jpg")), (224, 224)), axis = 0)

octaves = [{
    'scale': 1.0,
    'iter_n': 190,
    'start_sigma': .44,
    'end_sigma': 0.304,
    }, {
    'scale': 1.2,
    'iter_n': 150,
    'start_sigma': 0.44,
    'end_sigma': 0.304,
    }, {
    'scale': 1.2,
    'iter_n': 150,
    'start_sigma': 0.44,
    'end_sigma': 0.304,
    }, {
    'scale': 1.0,
    'iter_n': 10,
    'start_sigma': 0.44,
    'end_sigma': 0.304,
    }]

In [6]:
# import graph again so tf.default_graph() fetches the correct graph for visualization
tf.import_graph_def(graph_def, {'input':t_preprocessed})

# deepdream visualization
layer = "import/softmax2_pre_activation"

background_color = np.float32([10, 90, 140])

start = time.time()
# api call
is_success = class_visualization(graph_or_path = tf.get_default_graph(), value_feed_dict = {t_input : im}, 
                                     layer=layer, classes = [2],path_logdir="./Log/Inception5", 
                                     path_outdir="./Output/Inception5", background_color=background_color, octaves=octaves, use_bilateral=True)
start = time.time() - start
print("Total Time = %f" % (start))

INFO:tensorflow:Restoring parameters from model/tmp-model


/home/s/anaconda2/envs/magenta/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


octave 0 | layer import/softmax2_pre_activation | scale 1.0 | iter_n 190
octave 1 | layer import/softmax2_pre_activation | scale 1.2 | iter_n 150
octave 2 | layer import/softmax2_pre_activation | scale 1.2 | iter_n 150
octave 3 | layer import/softmax2_pre_activation | scale 1.0 | iter_n 10
2 -> featuremap completed.
Reconstruction Completed for import/softmax2_pre_activation layer. Time taken = 207.266023 s
Total Time = 208.090401
